# Ollama Setup and Testing

This notebook will help you set up and test Ollama with LangChain connectors before starting the main RAG assignment.

## Prerequisites

1. **Install Ollama** from https://ollama.ai
   - On Linux/Mac: `curl https://ollama.ai/install.sh | sh`
   - On Windows: Download and run the installer

2. **Verify Installation**
   - Run `ollama -v` in your terminal
   - Should show version 0.11.10 or greater

3. **Pull Required Models**
   ```bash
   # For chat/inference
   ollama pull gpt-oss:20b
   
   # For embeddings
   ollama pull embeddinggemma:latest

   # Pulled the smaller model as I was facing some memory issues
   ollama pull llama3.2:3b
   ```


## Step 1: Test Ollama Connection

First, let's verify that Ollama is running and accessible:


In [2]:
import requests
import json

# Test if Ollama is running
try:
    response = requests.get('http://localhost:11434/api/tags')
    if response.status_code == 200:
        models = json.loads(response.text)
        print("✅ Ollama is running!")
        print("\nAvailable models:")
        for model in models.get('models', []):
            print(f"  - {model['name']}")
    else:
        print("❌ Ollama is not responding properly")
except requests.exceptions.ConnectionError:
    print("❌ Cannot connect to Ollama. Make sure it's running!")
    print("Start Ollama by running 'ollama serve' in a terminal")


✅ Ollama is running!

Available models:
  - llama3.2:3b
  - gpt-oss:20b
  - embeddinggemma:latest


## Step 2: Test Embeddings with Ollama

Now let's test creating embeddings using the LangChain Ollama connector:


In [ ]:
from langchain_ollama import OllamaEmbeddings

# Initialize the embedding model
embedding_model = OllamaEmbeddings(
    model="embeddinggemma:latest",
    base_url="http://localhost:11434"  # Default Ollama URL
)

print("✅ Embedding model initialized")

✅ Embedding model initialized


In [ ]:
# Test embedding a single query
test_query = "What is the meaning of life?"

print(f"Embedding query: '{test_query}'")
embedding = embedding_model.embed_query(test_query)

print(f"\n✅ Successfully created embedding!")
print(f"Embedding dimension: {len(embedding)}")
print(f"First 10 values: {embedding[:10]}")


Embedding query: 'What is the meaning of life?'

✅ Successfully created embedding!
Embedding dimension: 768
First 10 values: [-0.14623907, 0.029114315, 0.037601642, -0.024860876, -0.026523322, 0.016056035, -0.027474474, 0.027252708, 0.011388898, -5.286664e-06]


In [ ]:
# Test embedding multiple documents
test_documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is a subset of artificial intelligence.",
    "Python is a popular programming language for data science."
]

print("Embedding multiple documents...")
embeddings = embedding_model.embed_documents(test_documents)

print(f"\n✅ Successfully created {len(embeddings)} embeddings!")
for i, doc in enumerate(test_documents):
    print(f"\nDocument {i+1}: '{doc[:50]}...'")
    print(f"  Embedding dimension: {len(embeddings[i])}")
    print(f"  First 5 values: {embeddings[i][:5]}")


Embedding multiple documents...

✅ Successfully created 3 embeddings!

Document 1: 'The quick brown fox jumps over the lazy dog....'
  Embedding dimension: 768
  First 5 values: [-0.14782467, 0.0028958742, 0.05214771, -0.029095568, -0.03669252]

Document 2: 'Machine learning is a subset of artificial intelli...'
  Embedding dimension: 768
  First 5 values: [-0.124075405, -0.0027447406, -0.00032587492, 0.010075077, 0.0016823779]

Document 3: 'Python is a popular programming language for data ...'
  Embedding dimension: 768
  First 5 values: [-0.16268292, -0.010286801, 0.02545166, 0.000692921, -0.018880809]


## Step 3: Test Model Inference with Ollama

Now let's test using Ollama for text generation/inference using the LangChain connector:


In [ ]:
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

# Initialize the chat model
chat_model = ChatOllama(
    model="gpt-oss:20b",
    temperature=0.6,
    base_url="http://localhost:11434",
    verbose=True
)

print("✅ Chat model initialized")


✅ Chat model initialized


#Let's add a procedure to measure the inference performance of the model

In [ ]:
def detailed_performance_metrics(response_metadata):
    """
    Calculate comprehensive performance metrics from Ollama response metadata
    """
    # Extract all timing data (in nanoseconds)
    total_duration = response_metadata.get('total_duration', 0)
    load_duration = response_metadata.get('load_duration', 0)
    prompt_eval_duration = response_metadata.get('prompt_eval_duration', 0)
    eval_duration = response_metadata.get('eval_duration', 0)
    
    # Extract token counts
    prompt_eval_count = response_metadata.get('prompt_eval_count', 0)
    eval_count = response_metadata.get('eval_count', 0)
    
    # Convert to seconds
    total_seconds = total_duration / 1_000_000_000
    load_seconds = load_duration / 1_000_000_000
    prompt_eval_seconds = prompt_eval_duration / 1_000_000_000
    eval_seconds = eval_duration / 1_000_000_000
    
    # tokens per second
    tokens_per_second = eval_count / eval_seconds

    # Calculate metrics
    metrics = {
        'generation_tokens_per_second': eval_count / eval_seconds if eval_seconds > 0 else 0,
        'prompt_tokens_per_second': prompt_eval_count / prompt_eval_seconds if prompt_eval_seconds > 0 else 0,
        'total_tokens': prompt_eval_count + eval_count,
        'total_time_seconds': total_seconds,
        'load_time_seconds': load_seconds,
        'generation_time_seconds': eval_seconds,
        'prompt_processing_time_seconds': prompt_eval_seconds,
        }

    print(f"Total tokens: {metrics['total_tokens']}")
    print(f"Total time seconds: {metrics['total_time_seconds']}")
    print(f"Load time seconds: {metrics['load_time_seconds']}")
    print(f"Generation time seconds: {metrics['generation_time_seconds']}")
    print(f"Prompt processing time seconds: {metrics['prompt_processing_time_seconds']}")
    print(f"Generation tokens per second: {metrics['generation_tokens_per_second']}")
    print(f"Prompt tokens per second: {metrics['prompt_tokens_per_second']}")

    return metrics

In [ ]:
# Test simple inference
prompt = "Explain quantum computing in one sentence."

print(f"Prompt: {prompt}")
print("\nGenerating response...")

response = chat_model.invoke(prompt)

print(f"\n✅ Response generated!")
print(f"\nModel output: {response.content}")


In [ ]:
_ = detailed_performance_metrics(response.response_metadata)

Total tokens: 191
Total time seconds: 3757.006019083
Load time seconds: 57.329620458
Generation time seconds: 3599.344005375
Prompt processing time seconds: 99.879633
Generation tokens per second: 0.0325059232530374
Prompt tokens per second: 0.7408917892199304


In [ ]:
# Test with system message and human message
messages = [
    SystemMessage(content="You are a helpful AI assistant that explains complex topics simply."),
    HumanMessage(content="What is machine learning?")
]

print("Sending messages to model...")
response = chat_model.invoke(messages)

print(f"\n✅ Response generated!")
print(f"\nModel output: {response.content}")


Sending messages to model...



✅ Response generated!

Model output: ### Machine Learning in a nutshell

**Machine learning (ML)** is a way to let computers learn from data instead of being told every single step by a human programmer.

---

#### 1. The basic idea

| Step | What happens | Analogy |
|------|--------------|---------|
| **Collect data** | Gather examples (images, text, numbers, etc.) | A teacher gives a student many practice problems. |
| **Teach the model** | Use math and algorithms to find patterns in the data | The student studies the solutions and notices patterns. |
| **Make predictions** | The trained model can now guess the answer for new, unseen data | The student solves a new problem using the patterns learned. |

So, instead of writing a rule‑by‑rule program, we give the computer a lot of examples and let it figure out the rule itself.

---

#### 2. Why it matters

- **Automation**: Tasks that would take humans hours can be done in milliseconds (e.g., recognizing faces in photos).
- **Adaptab

In [ ]:
_ = detailed_performance_metrics(response.response_metadata)

Total tokens: 743
Total time seconds: 1065.684481166
Load time seconds: 2.131715291
Generation time seconds: 1024.91029675
Prompt processing time seconds: 37.508599917
Generation tokens per second: 0.635177539014221
Prompt tokens per second: 2.452770836650261


## Step 4: Test Streaming Response

Ollama supports streaming responses, which is useful for real-time applications:


In [ ]:
# Test streaming
prompt = "Write a haiku about artificial intelligence."

print(f"Prompt: {prompt}")
print("\nStreaming response:")
print("-" * 40)

for chunk in chat_model.stream(prompt):
    print(chunk.content, end="", flush=True)

print("\n" + "-" * 40)
print("\n✅ Streaming completed!")


Prompt: Write a haiku about artificial intelligence.

Streaming response:
----------------------------------------


Silent circuits hum,  
Minds of code, dreaming in light—  
Future’s quiet pulse.
----------------------------------------

✅ Streaming completed!


## Summary

If all the tests above passed, you're ready to use Ollama with LangChain! Here's what we tested:

✅ **Embeddings**: 
- Created embeddings for single queries
- Created embeddings for multiple documents
- Verified embedding dimensions

✅ **Model Inference**:
- Simple text generation
- Chat with system and human messages
- Streaming responses
- Integration with LangChain chains

## Troubleshooting

If you encounter issues:

1. **Model Not Found**: Pull the required models (`ollama pull <model-name>`)
2. **Slow Performance**: Ollama models run on CPU by default. For better performance:
   - Use smaller models for testing
   - Consider GPU acceleration if available
3. **Memory Issues**: Large models require significant RAM. Try smaller variants if needed.

## Next Steps

Now you're ready to proceed with the main RAG assignment using Ollama!
